# Runge-Kutta Fourth Order Method for System of ODEs / 2nd Order ODE

## Working Principle
The RK4 method is an iterative method for solving ordinary differential equations (ODEs). For a system of ODEs or higher-order ODEs, they are reduced to a set of first-order ODEs, and the RK4 method is applied to each equation simultaneously.

## Pseudocode

### Case A: System of ODEs

1. **Define the system of ODEs:**
   - dy₁/dx = f₁(x, y₁, y₂, ..., yₙ)
   - dy₂/dx = f₂(x, y₁, y₂, ..., yₙ)
   - ... (Repeat for all equations in the system)

2. **Set initial values** for (x, y₁, y₂, ..., yₙ)

3. **For each step i, compute:**
   - k1ᵢ = h · fᵢ(x, y₁, ..., yₙ)
   - k2ᵢ = h · fᵢ(x + h/2, y₁ + k1₁/2, ..., yₙ + k1ₙ/2)
   - k3ᵢ = h · fᵢ(x + h/2, y₁ + k2₁/2, ..., yₙ + k2ₙ/2)
   - k4ᵢ = h · fᵢ(x + h, y₁ + k3₁, ..., yₙ + k3ₙ)

4. **Update each variable:**
   - yᵢ = yᵢ + (1/6)(k1ᵢ + 2k2ᵢ + 2k3ᵢ + k4ᵢ)

5. **Increment x by h** and repeat until the desired xₑₙd is reached

### Case B: Second-Order ODE

1. **Convert the second-order ODE into a system of two first-order ODEs.**
   
   For example: d²y/dx² = f(x, y, dy/dx)
   
   Becomes:
   - dy₁/dx = y₂
   - dy₂/dx = f(x, y₁, y₂)
   
   Where: y₁ = y and y₂ = dy/dx

2. **Solve the system of ODEs** using the RK4 method as outlined above

In [1]:
import math

def rk4_system(functions, x0, y0_list, h, x_end):
    """
    Solve a system of first-order ODEs using RK4 method
    
    Parameters:
    functions: list of functions [f1, f2, ..., fn] where fi(x, y_list) returns dy_i/dx
    x0: initial x value
    y0_list: list of initial y values [y1_0, y2_0, ..., yn_0]
    h: step size
    x_end: final x value
    
    Returns:
    x_values: list of x values
    y_values: list of lists, each containing y_i values
    """
    n = len(functions)
    x_values = [x0]
    y_values = [[y0_list[i]] for i in range(n)]
    
    x = x0
    y = y0_list[:]
    
    while x < x_end:
        # Calculate k1 for all variables
        k1 = [h * functions[i](x, y) for i in range(n)]
        
        # Calculate k2 for all variables
        y_temp = [y[i] + k1[i]/2 for i in range(n)]
        k2 = [h * functions[i](x + h/2, y_temp) for i in range(n)]
        
        # Calculate k3 for all variables
        y_temp = [y[i] + k2[i]/2 for i in range(n)]
        k3 = [h * functions[i](x + h/2, y_temp) for i in range(n)]
        
        # Calculate k4 for all variables
        y_temp = [y[i] + k3[i] for i in range(n)]
        k4 = [h * functions[i](x + h, y_temp) for i in range(n)]
        
        # Update all y values
        for i in range(n):
            y[i] = y[i] + (1/6) * (k1[i] + 2*k2[i] + 2*k3[i] + k4[i])
        
        # Update x
        x = x + h
        
        # Store values
        x_values.append(x)
        for i in range(n):
            y_values[i].append(y[i])
    
    return x_values, y_values

def rk4_second_order(f, x0, y0, dy0, h, x_end):
    """
    Solve second-order ODE: d²y/dx² = f(x, y, dy/dx)
    
    Parameters:
    f: function f(x, y, dy_dx) representing d²y/dx²
    x0: initial x value
    y0: initial y value
    dy0: initial dy/dx value
    h: step size
    x_end: final x value
    
    Returns:
    x_values: list of x values
    y_values: list of y values
    dy_values: list of dy/dx values
    """
    # Convert to system of first-order ODEs
    def f1(x, vars):  # dy1/dx = y2
        return vars[1]
    
    def f2(x, vars):  # dy2/dx = f(x, y1, y2)
        return f(x, vars[0], vars[1])
    
    functions = [f1, f2]
    x_vals, y_vals = rk4_system(functions, x0, [y0, dy0], h, x_end)
    
    return x_vals, y_vals[0], y_vals[1]

# Example 1: System of ODEs - Predator-Prey (Lotka-Volterra)
print("Example 1: Predator-Prey System")
print("dx/dt = ax - bxy")
print("dy/dt = -cy + dxy")

def predator_prey_x(t, vars):  # dx/dt = ax - bxy
    x, y = vars[0], vars[1]
    a, b = 1.0, 0.5
    return a * x - b * x * y

def predator_prey_y(t, vars):  # dy/dt = -cy + dxy
    x, y = vars[0], vars[1]
    c, d = 0.75, 0.25
    return -c * y + d * x * y

functions = [predator_prey_x, predator_prey_y]
t0, x0, y0 = 0, 2, 1  # Initial: t=0, prey=2, predator=1
h = 0.1
t_end = 10

t_vals, populations = rk4_system(functions, t0, [x0, y0], h, t_end)

print("Time\tPrey\tPredator")
print("-" * 30)
for i in range(0, len(t_vals), 20):  # Print every 20th value
    print(f"{t_vals[i]:.1f}\t{populations[0][i]:.4f}\t{populations[1][i]:.4f}")

print("\n" + "="*60 + "\n")

# Example 2: Second-order ODE - Simple Harmonic Oscillator
print("Example 2: Simple Harmonic Oscillator")
print("d²y/dx² + y = 0, y(0) = 1, dy/dx(0) = 0")

def harmonic_oscillator(x, y, dy_dx):
    return -y  # d²y/dx² = -y

x0, y0, dy0 = 0, 1, 0
h = 0.1
x_end = 2 * math.pi

x_vals, y_vals, dy_vals = rk4_second_order(harmonic_oscillator, x0, y0, dy0, h, x_end)

print("x\tRK4 y\tExact y\tError")
print("-" * 40)
for i in range(0, len(x_vals), 16):  # Print selected values
    x = x_vals[i]
    y_rk4 = y_vals[i]
    y_exact = math.cos(x)  # Exact solution: y = cos(x)
    error = abs(y_exact - y_rk4)
    print(f"{x:.2f}\t{y_rk4:.6f}\t{y_exact:.6f}\t{error:.8f}")

print("\n" + "="*60 + "\n")

# Example 3: Second-order ODE - Damped Oscillator
print("Example 3: Damped Oscillator")
print("d²y/dx² + 2dy/dx + 5y = 0, y(0) = 1, dy/dx(0) = 0")

def damped_oscillator(x, y, dy_dx):
    return -2 * dy_dx - 5 * y

x0, y0, dy0 = 0, 1, 0
h = 0.05
x_end = 3

x_vals, y_vals, dy_vals = rk4_second_order(damped_oscillator, x0, y0, dy0, h, x_end)

print("x\ty\tdy/dx")
print("-" * 25)
for i in range(0, len(x_vals), 20):  # Print every 20th value
    print(f"{x_vals[i]:.2f}\t{y_vals[i]:.6f}\t{dy_vals[i]:.6f}")

print("\n" + "="*60 + "\n")

# Example 4: System of 3 ODEs - Lorenz System (simplified)
print("Example 4: 3-Variable System")
print("dx/dt = -ax + ay")
print("dy/dt = rx - y - xz")  
print("dz/dt = xy - bz")

def lorenz_x(t, vars):
    x, y, z = vars[0], vars[1], vars[2]
    a = 10
    return -a * x + a * y

def lorenz_y(t, vars):
    x, y, z = vars[0], vars[1], vars[2]
    r = 28
    return r * x - y - x * z

def lorenz_z(t, vars):
    x, y, z = vars[0], vars[1], vars[2]
    b = 8/3
    return x * y - b * z

functions = [lorenz_x, lorenz_y, lorenz_z]
t0 = 0
initial_conditions = [1, 1, 1]
h = 0.01
t_end = 2

t_vals, vars = rk4_system(functions, t0, initial_conditions, h, t_end)

print("Time\tx\ty\tz")
print("-" * 35)
for i in range(0, len(t_vals), 50):  # Print every 50th value
    print(f"{t_vals[i]:.2f}\t{vars[0][i]:.4f}\t{vars[1][i]:.4f}\t{vars[2][i]:.4f}")

print("\n" + "="*60 + "\n")

# Example 5: Second-order with forcing - driven oscillator
print("Example 5: Driven Oscillator")
print("d²y/dx² + y = cos(2x), y(0) = 0, dy/dx(0) = 0")

def driven_oscillator(x, y, dy_dx):
    return -y + math.cos(2 * x)

x0, y0, dy0 = 0, 0, 0
h = 0.05
x_end = 4 * math.pi

x_vals, y_vals, dy_vals = rk4_second_order(driven_oscillator, x0, y0, dy0, h, x_end)

print("x\ty\tdy/dx")
print("-" * 25)
for i in range(0, len(x_vals), 25):  # Print every 25th value
    print(f"{x_vals[i]:.2f}\t{y_vals[i]:.6f}\t{dy_vals[i]:.6f}")

Example 1: Predator-Prey System
dx/dt = ax - bxy
dy/dt = -cy + dxy
Time	Prey	Predator
------------------------------
0.0	2.0000	1.0000
2.0	5.4887	1.3100
4.0	2.7967	3.7459
6.0	1.1815	1.8488
8.0	2.4526	0.9260
10.0	6.0529	1.7244


Example 2: Simple Harmonic Oscillator
d²y/dx² + y = 0, y(0) = 1, dy/dx(0) = 0
x	RK4 y	Exact y	Error
----------------------------------------
0.00	1.000000	1.000000	0.00000000
1.60	-0.029198	-0.029200	0.00000133
3.20	-0.998295	-0.998295	0.00000007
4.80	0.087495	0.087499	0.00000400


Example 3: Damped Oscillator
d²y/dx² + 2dy/dx + 5y = 0, y(0) = 1, dy/dx(0) = 0
x	y	dy/dx
-------------------------
0.00	1.000000	0.000000
1.00	0.014163	-0.836281
2.00	-0.139673	0.256057
3.00	0.040849	0.034778


Example 4: 3-Variable System
dx/dt = -ax + ay
dy/dt = rx - y - xz
dz/dt = xy - bz
Time	x	y	z
-----------------------------------
0.00	1.0000	1.0000	1.0000
0.50	1.1986	-8.8671	32.4549
1.00	-9.3786	-8.3571	29.3624
1.50	-9.6723	-10.4320	27.5174
2.00	-8.1734	-9.5620	24.6206


Examp